In [8]:
import pandas as pd
import time
import os
from selenium import webdriver

from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

It has a series of steps that you need to scrap:
- open the chrome driver
- open the linkedin jobs url



In [74]:
url = "https://www.linkedin.com/jobs/search/?currentJobId=3477276853&geoId=106677219&keywords=data%20analyst&location=Waterloo%2C%20Ontario%2C%20Canada&refresh=true"
driver_path = "./chromedriver"

driver = webdriver.Chrome(executable_path=driver_path)
driver.get(url)
driver.implicitly_wait(5)

In [75]:
def update_searchbar(job_titles,location):
    # locate the search bar
    searchbar_job = driver.find_element_by_name("keywords")
    searchbar_location = driver.find_element_by_name("location")
    # clear the default text
    searchbar_job.clear()
    searchbar_location.clear()
    # send in newtext
    searchbar_job.send_keys(job_titles)
    searchbar_location.send_keys(location)

    time.sleep(2)
    # press enter in the search bar
    searchbar_job.send_keys(Keys.RETURN)

update_searchbar(job_titles="data engineer",location="toronto, ontario, canada")

Now we need to add the filtering condition
- any time
- location
- experience level

In [76]:
def apply_job_filtering(button_css,filter_condition_css,done_css):
    """
    apply the job filtering condition based on css selector of the web elements.
    You need to use CSS selector to select it
    Args:
        button_css (_type_): _description_
        filter_condition_css (_type_): _description_
        done_css (_type_): _description_
    """
    # locate and click the filter time button
    button_filter_time = driver.find_element_by_css_selector(button_css)
    driver.execute_script("arguments[0].click();",button_filter_time)
    time.sleep(1)
    # locate and click the past 24 hours
    filter_time_past_24 = driver.find_element_by_css_selector(filter_condition_css)
    driver.execute_script("arguments[0].click();",filter_time_past_24)
    time.sleep(1)
    # locate and click the "done" button
    button_done = driver.find_element_by_css_selector(done_css)
    driver.execute_script("arguments[0].click();",button_done)
    time.sleep(1)

# apply date filtering to the past month
apply_job_filtering(button_css = "form#jserp-filters li:nth-child(1) > div > div > button",
                    filter_condition_css = "form#jserp-filters li:nth-child(1) > div > div > div > div > div > div:nth-child(3) > label",
                    done_css= "form#jserp-filters li:nth-child(1) > div > div > div > button")


Now you need to infintely scroll down the pages and click load more

In [77]:
def scroll_down(number_of_iterations = 50):
    """
    indefinitely scroll down until no jobs could be loaded
    Args:
        number_of_iterations (_type_): _description_
    """
    # i dummy iterator, the program time is actually controled by the time.sleep()
    i = 0
    while i <= number_of_iterations:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        i += 1

        try:
            # find th button
            button_see_jobs = driver.find_element_by_class_name("infinite-scroller__show-more-button")        
            # click it
            driver.execute_script("arguments[0].click();",button_see_jobs)
            
            #3 secs grace period for it to load
            time.sleep(2)
        except:
            # not finding the button, wait for three secs and continue
            pass
            time.sleep(1)

scroll_down()

Now you should have scroll down to the very bottom of the pages with no more jobs available.

In [85]:
company_name = []
job_titles = []
links = []

for i in range(300):
    try:
        # grab the necessary info
        link = driver.find_elements_by_class_name('hidden-nested-link')[i].get_attribute('href')
        # link = driver.find_elements_by_class_name('base-card__full-link')[i].get_attribute('href')

        job_title = driver.find_elements_by_class_name('base-search-card__title')[i].text
        company = driver.find_elements_by_class_name('base-search-card__subtitle')[i].text
        # append it to a list
        company_name.append(company)
        job_titles.append(job_title)
        links.append(link)
    except IndexError:
        print("job done!")



In [86]:
df = pd.DataFrame()
df["company"] = company_name
df["job_title"] = job_titles
df["link"] = links

df.head()

,company,job_title,link
0,Lyft,Data Engineer,https://ca.linkedin.com/jobs/view/data-enginee...
1,Think Research,Data Engineer,https://ca.linkedin.com/jobs/view/data-enginee...
2,Borrowell,Senior Data Engineer,https://ca.linkedin.com/jobs/view/senior-data-...
3,Points (a Plusgrade company),Data Engineer,https://ca.linkedin.com/jobs/view/data-enginee...
4,The Credit Pros,Data Engineer,https://ca.linkedin.com/jobs/view/data-enginee...


It is still has the problems that some link are missing. Instead of pulling job title from main page, we should just scrap the link and scrape your company, jobtitle information there. It should solve the problem.

In [90]:

df.to_csv("./test.csv")